# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 2:22PM,247409,10,Eme-104610,Emerginnova,Released
1,Sep 27 2022 2:21PM,247408,22,628752,"NBTY Global, Inc.",Released
2,Sep 27 2022 2:21PM,247407,22,628750,"NBTY Global, Inc.",Released
3,Sep 27 2022 2:14PM,247406,10,WT5354 Out,Eywa Pharma Inc.,Released
4,Sep 27 2022 2:08PM,247405,10,0086015836,ISDIN Corporation,Released
5,Sep 27 2022 2:08PM,247405,10,0086015840,ISDIN Corporation,Released
6,Sep 27 2022 2:08PM,247405,10,0086015841,ISDIN Corporation,Released
7,Sep 27 2022 2:08PM,247405,10,0086015839,ISDIN Corporation,Released
8,Sep 27 2022 2:08PM,247405,10,0086015842,ISDIN Corporation,Released
9,Sep 27 2022 2:08PM,247405,10,0086015845,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,247405,Released,51
18,247406,Released,1
19,247407,Released,1
20,247408,Released,1
21,247409,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247405,NaN,NaN,51.0
247406,NaN,NaN,1.0
247407,NaN,NaN,1.0
247408,NaN,NaN,1.0
247409,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0.0,0.0,1.0
247205,0.0,1.0,0.0
247331,1.0,0.0,2.0
247332,0.0,0.0,1.0
247345,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0,0,1
247205,0,1,0
247331,1,0,2
247332,0,0,1
247345,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,0,0,1
1,247205,0,1,0
2,247331,1,0,2
3,247332,0,0,1
4,247345,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,,,1
1,247205,,1,
2,247331,1,,2
3,247332,,,1
4,247345,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 2:22PM,247409,10,Emerginnova
1,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc."
2,Sep 27 2022 2:21PM,247407,22,"NBTY Global, Inc."
3,Sep 27 2022 2:14PM,247406,10,Eywa Pharma Inc.
4,Sep 27 2022 2:08PM,247405,10,ISDIN Corporation
55,Sep 27 2022 2:06PM,247403,20,"ACI Healthcare USA, Inc."
84,Sep 27 2022 2:01PM,247404,10,ISDIN Corporation
98,Sep 27 2022 1:55PM,247401,10,Yusen – 3D Matrix
99,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc."
101,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 2:22PM,247409,10,Emerginnova,,,1
1,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",,,1
2,Sep 27 2022 2:21PM,247407,22,"NBTY Global, Inc.",,,1
3,Sep 27 2022 2:14PM,247406,10,Eywa Pharma Inc.,,,1
4,Sep 27 2022 2:08PM,247405,10,ISDIN Corporation,,,51
5,Sep 27 2022 2:06PM,247403,20,"ACI Healthcare USA, Inc.",,,29
6,Sep 27 2022 2:01PM,247404,10,ISDIN Corporation,,,14
7,Sep 27 2022 1:55PM,247401,10,Yusen – 3D Matrix,,,1
8,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",,,2
9,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 2:22PM,247409,10,Emerginnova,1,,
1,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",1,,
2,Sep 27 2022 2:21PM,247407,22,"NBTY Global, Inc.",1,,
3,Sep 27 2022 2:14PM,247406,10,Eywa Pharma Inc.,1,,
4,Sep 27 2022 2:08PM,247405,10,ISDIN Corporation,51,,
5,Sep 27 2022 2:06PM,247403,20,"ACI Healthcare USA, Inc.",29,,
6,Sep 27 2022 2:01PM,247404,10,ISDIN Corporation,14,,
7,Sep 27 2022 1:55PM,247401,10,Yusen – 3D Matrix,1,,
8,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",2,,
9,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 2:22PM,247409,10,Emerginnova,1,,
1,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",1,,
2,Sep 27 2022 2:21PM,247407,22,"NBTY Global, Inc.",1,,
3,Sep 27 2022 2:14PM,247406,10,Eywa Pharma Inc.,1,,
4,Sep 27 2022 2:08PM,247405,10,ISDIN Corporation,51,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 2:22PM,247409,10,Emerginnova,1.0,NaN,NaN
1,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Sep 27 2022 2:21PM,247407,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Sep 27 2022 2:14PM,247406,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Sep 27 2022 2:08PM,247405,10,ISDIN Corporation,51.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 2:22PM,247409,10,Emerginnova,1.0,0.0,0.0
1,Sep 27 2022 2:21PM,247408,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Sep 27 2022 2:21PM,247407,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Sep 27 2022 2:14PM,247406,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Sep 27 2022 2:08PM,247405,10,ISDIN Corporation,51.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2721131,130.0,0.0,1.0
15,247352,7.0,60.0,0.0
20,742174,52.0,1.0,0.0
21,494407,1.0,1.0,0.0
22,494815,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2721131,130.0,0.0,1.0
1,15,247352,7.0,60.0,0.0
2,20,742174,52.0,1.0,0.0
3,21,494407,1.0,1.0,0.0
4,22,494815,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,130.0,0.0,1.0
1,15,7.0,60.0,0.0
2,20,52.0,1.0,0.0
3,21,1.0,1.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,130.0
1,15,Released,7.0
2,20,Released,52.0
3,21,Released,1.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,20,21,22
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,0.0,60.0,1.0,1.0,0.0
Released,130.0,7.0,52.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,60.0,1.0,1.0,0.0
2,Released,130.0,7.0,52.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,60.0,1.0,1.0,0.0
2,Released,130.0,7.0,52.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()